# GDB LE5: Nicht-relationale Datenbanken lesen

## Aufgabenstellung
Du solltest ein Datenset auswählen, der **besser** in einer NoSQL Datenbank (als in einer relationalen) passt und dich mit der Datenbank und mit den Daten auseinandersetzen.

Damit wir Deinen Weg und Lernerfolg einschätzen können benötigen wir von Dir eine kurze Zusammenfassung.

In dieser Zusammenfassung sollen folgende Fragen beantwortet werden:

1. Wie sehen die Daten aus?
2. Welche Datenbank hast Du gewählt?
3. Warum hast Du diese gewählt?
4. Inwiefern ist das besser als eine relationale Datenbank?
5. Wie sehen komplexe Fragestellungen (Abfrage) zu den Daten aus, und warum sind sie komplex? Vergleiche es mit eine SQL, würde es komplizierter sein, welche Vorteile gibt es gegenüber SQL?

Bevor wir diese Fragen beantworten können, müssen wir die Verbindung zur Datenbank erstellen.

### Vorbereitung

In [5]:
from pymongo import MongoClient
import urllib
import pandas
import configparser
import pprint

config = configparser.ConfigParser()
config.read('config.ini')

db_username = config["MongoDB"]["username"]
db_password = config["MongoDB"]["password"]
db_hostname = config["MongoDB"]["hostname"]
db_link = config["MongoDB"]["link"]

jsonp = pprint.pprint

In [6]:
# IP Adresse ausgeben
!curl ipecho.net/plain

147.86.207.49

In [7]:
# Verbindung zur MongoDB Atlas
client = MongoClient('mongodb+srv://{}:{}@{}.{}.mongodb.net/sample_analytics'.format(db_username, db_password, db_hostname, db_link))
db = client.sample_analytics

### Aufgabe 1 - Wie sehen die Daten aus?

Um unsere Daten besser analysieren zu können müssen wir die Daten zuerst verstehen.

In [42]:
# Collections der Datenbank ausgeben
db.list_collection_names()

['customers', 'transactions', 'accounts']

Hier sehen wir, dass die Datenbank "sample_analytics" 3 Tabellen hat:
- customers
- transactions
- accounts

Jetzt nehmen wir die einzelnen Tabellen genauer unter die Lupe, um zu schauen was für Daten beinhaltet sind. Dafür schreiben wir eine Methode, die alle Attribute der Tabelle zu einer Liste hinzufügt. Als Parameter wird die einzelnen Tabellen übergeben

In [58]:
def check_table(cursor):
    attributes = []
    
    # Attribute in der Tabelle ausgeben
    for document in cursor.find():
        for i in document:
            if i not in attributes:
                attributes.append(i)
                
    return attributes

In [61]:
print("Attribute von Accounts: {}".format(check_table(db.accounts)))

print("Attribute von Customers: {}".format(check_table(db.customers)))

print("Attribute von Transactions: {}".format(check_table(db.transactions)))

Attribute von Accounts: ['_id', 'account_id', 'limit', 'products']
Attribute von Customers: ['_id', 'username', 'name', 'address', 'birthdate', 'email', 'active', 'accounts', 'tier_and_details']
Attribute von Transactions: ['_id', 'account_id', 'transaction_count', 'bucket_start_date', 'bucket_end_date', 'transactions']


### Aufgabe 2 - Welche Datenbank hast Du gewählt?

Für die NoSQL Datenbank haben wir uns für MongoDB entschieden.

### Aufgabe 3 - Warum hast Du diese gewählt?

Analytics, weil...

### Aufgabe 4 - Inwiefern ist das besser als eine relationale Datenbank?

### Aufgabe 5 - Wie sehen komplexe Fragestellungen zu den Daten aus?

#### Warum sind sie komplex?

#### Vergleich mit SQL

#### Vorteile gegenüber SQL?

### Fragen:

1. Wie viele Kunden haben eine Limite unter 10000?
2. Wie viele Kunden haben "Brokerage"?
4. Was für Produkte hat Kunde '446747'?
3. Was ist das meist vorkommende Produkt?

In [59]:
# "Accounts" Collection auswählen
cursor = db.accounts

In [17]:
# Frage 1

count = 0
for i in cursor.find({"limit": {"$lt": 10000}}):
    count += 1
    
print(count)

45


In [18]:
# Frage 1 Aggregate

result = accounts.aggregate([{"$match": { "$expr": { "$gt": [10000, "$limit"]}}}, {"$count": "unter 10000"}])

for res in result:
    print(res)

{'unter 10000': 45}


In [19]:
# Frage 2

count = 0
for res in cursor.find({"products": "Brokerage"}):
    count += 1

print(count)

741


In [20]:
# Frage 2 Aggregate

result = accounts.aggregate([{"$match": {"products": "Brokerage"}}, {"$count": "Brokerage"}])

for res in result:
    print(res)

{'Brokerage': 741}


In [35]:
# Frage 3

results = cursor.find({"account_id": 446747})

for result in results:
    print(result)
    for x in result["products"]:
        print("- {}".format(x))

NameError: name 'cursor' is not defined

In [22]:
# Frage 3 Aggregate

result = accounts.aggregate([{"$match": {"account_id": 446747}}])

for res in result:
    print(res["products"])

['Derivatives', 'Commodity', 'Brokerage', 'InvestmentStock']


In [23]:
# Frage 4 (Hard coded)

print("Was ist das meist vorkommende Produkt?")

result = accounts.find()

der_count = 0
inv_s_count = 0
comm_count = 0
broke_count = 0
curr_s = 0
inv_f_count = 0

for res in result:
    if "Derivatives" in res["products"]:
        der_count += 1
    if "InvestmentStock" in res["products"]:
        inv_s_count += 1
    if "Commodity" in res["products"]:
        comm_count += 1
    if "Brokerage" in res["products"]:
        broke_count += 1
    if "CurrencyService" in res["products"]:
        curr_s += 1
    if "InvestmentFund" in res["products"]:
        inv_f_count += 1

print(der_count)
print(inv_s_count)
print(comm_count)
print(broke_count)
print(curr_s)
print(inv_f_count)

Was ist das meist vorkommende Produkt?
706
1746
720
741
742
728


### Fragen über Customers:

1. Was ist die Durchschnittsalter der Kunden?
2. ...
3. ...

In [24]:
cursor = db.customers

In [25]:
# Attribute in der Tabelle "customers" ausgeben
for document in cursor.find():
    for i in document:
        print(i)

_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accoun

tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birth

name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_d

tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birthdate
email
accounts
tier_and_details
_id
username
name
address
birth

Attribute:
- id
- username
- name
- address
- birthdate
- email
- active
- accounts
- tier_and_details

In [26]:
cursor = db.transactions

In [27]:
for document in cursor.find():
    for i in document:
        print(i)

_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_

transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_

transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
buck

transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_count
bucket_start_date
bucket_end_date
transactions
_id
account_id
transaction_

Attribute:

- id
- account id
- transaction count
- bucket start date
- bucket end date
- transactions

### MongoDB Einleitung (ignorieren)

Interestingly the result is not a collection: https://docs.mongodb.com/manual/reference/method/db.collection.find/index.html

In [34]:
for i1,res in enumerate(result):
  if i1%100==0:
    print(res)
print(i1)

1745


Now lets use mongodb to acquire the data we need. Here we will aggregate two collections, accounts with transactions. Further, we use the account_id from accounts and link to the foreignField account_id from transactions, similarly as "join on" in sql. We rename that transactions_link. We will also select only those records which have account_id less than 370000. 

In [12]:
result = accounts.aggregate([
    {
        '$lookup': {
            'from': 'transactions', 
            'localField': 'account_id', 
            'foreignField': 'account_id', 
            'as': 'transactions_link'
        }
    }, {
        '$match': {
            '$expr': {
                '$gt': [
                    370000, '$account_id'
                ]
            }
        }
    }
])

Let's check the result.

In [13]:
for res in result:
  print(res["account_id"],len(res["transactions_link"][0]["transactions"]))

328627 85
329562 99
139687 91
176639 22
77831 44
131176 54
246735 40
143720 79
285919 33
141237 45
107787 29
163269 65
261248 33
149440 8
120556 75
293516 5
226865 53
103536 48
54977 15
331043 98
300446 68
154391 4
155156 24
55104 78
165279 5
280867 83
151378 51
354107 4
135185 47
141597 3
130717 34
66698 75
103659 60
226114 21
111213 13
369055 64
303025 97
304914 65
161460 18
347313 54
170945 66
364643 29
185130 41
233104 70
230611 20
212024 32
353465 74
330318 72
118127 43
296866 96
130514 27
116390 2
162007 31
331169 58
51474 8
303316 99
66589 57
260499 94
137994 65
209363 56
240640 97
330961 27
126668 58
175894 96
348352 33
202669 61
156715 98
200611 14
183400 1
231223 16
136139 17
87191 26
165706 10
59715 93
264514 95
199711 69
88112 28
131885 35
312740 5
298562 36
271554 96
249109 58
155224 30
276528 47
278497 79
299100 52
126444 76
199559 21
160912 68
255695 17
323636 27
165436 27
155111 40
54368 32
122923 53
177069 25
109139 80
194523 3
77032 81
278603 83
198100 66
166084 73
15

Same story but we could the results.

In [14]:
result = accounts.aggregate([
    {
        '$lookup': {
            'from': 'transactions', 
            'localField': 'account_id', 
            'foreignField': 'account_id', 
            'as': 'transactions_link'
        }
    }, {
        '$match': {
            '$expr': {
                '$gt': [
                    370000, '$account_id'
                ]
            }
        }
    }, {
        '$count': 'account_id'
    }
])

In [15]:
for res in result:
  print(res)

{'account_id': 588}


If you want check the results by putting transactions and accounts into two different pandas dataframes and merging them with sql.
For the NoSQL-report the MongoDB query is needed. Using the syntax in the find is enough. You could double check using the pandas approach though.

**Weiterführende Fragen:**

Kannst du auswählen, welche account_id transactions mit weniger als 30 Elemente haben?
Welche account hat die älteste transactions?
Überlege noch 1-2 komplexe Fragen in diesem Datensatz.